In [1]:
library(Seurat)

library(dplyr)
library(glue)
library(ggplot2)
library(patchwork)
library(cowplot)
library(stringr)
library(tidyr)
library("dplyr")
library(velocyto.R)
library(stringr)
library(SeuratWrappers)
library(DAseq)
load("6_18.RData")

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘glue’


The following object is masked from ‘package:dplyr’:

    collapse



Attaching package: ‘cowplot’


The following object is masked from ‘package:patchwork’:

    align_plots


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




In [2]:
library('org.Mm.eg.db')
library(clusterProfiler)
library(dplyr)
library(tibble)
library(scales)


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, ta

In [3]:
df_list=list()
for (c in levels(merged_obj)){
    tmp = subset(x = merged_obj, subset = seurat_clusters == c)
    degs = FindMarkers(tmp, verbose=T,slot="data",ident.1 = "mNFIXko", ident.2 = "mWT",
            group.by="orig.ident",logfc.threshold = 0.05,test.used="wilcox",pseudocount.use=1)
    degs['fdr'] = p.adjust(degs$p_val, "BH")
    print (c)
    degs['Cluster']=c
    degs['geneName']=rownames(degs)
    rownames(degs) <- NULL
    df_list <- append(df_list, list(degs))
}

[1] "0"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "10"
[1] "11"
[1] "12"
[1] "13"
[1] "14"
[1] "15"
[1] "16"


In [4]:
all_genes=Reduce(rbind,df_list)

In [5]:
dim(all_genes)

[1] 58066     8

In [6]:
head(all_genes)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,fdr,Cluster,geneName
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,1.731954e-111,0.6266801,1.000,1.000,5.378238e-107,2.093933e-108,0,Gm42418
2,1.036221e-89,-0.5924145,0.394,0.772,3.217778e-85,6.263959e-87,0,Gm47283
3,6.458310e-79,-0.2286484,0.998,0.999,2.005499e-74,2.602699e-76,0,Rpl35a
4,3.027118e-73,-0.5725872,0.970,0.994,9.400110e-69,9.149465e-71,0,Alad
5,1.415365e-63,-0.1918138,0.998,1.000,4.395133e-59,3.422353e-61,0,Rpl41
6,2.715751e-62,-0.1987318,0.996,1.000,8.433223e-58,5.472239e-60,0,Rps15a


In [9]:
write.table(all_genes,"all_genes.for_volcano.tsv",sep="\t",quote = F,row.names=F)

In [8]:
?write.table

write.table {utils},R Documentation
x,"the object to be written, preferably a matrix or data frame. If not, it is attempted to coerce x to a data frame."
file,"either a character string naming a file or a connection open for writing. """" indicates output to the console."
append,"logical. Only relevant if file is a character string. If TRUE, the output is appended to the file. If FALSE, any existing file of the name is destroyed."
quote,"a logical value (TRUE or FALSE) or a numeric vector. If TRUE, any character or factor columns will be surrounded by double quotes. If a numeric vector, its elements are taken as the indices of columns to quote. In both cases, row and column names are quoted if they are written. If FALSE, nothing is quoted."
sep,the field separator string. Values within each row of x are separated by this string.
eol,"the character(s) to print at the end of each line (row). For example, eol = ""\r\n"" will produce Windows' line endings on a Unix-alike OS, and eol = ""\r"" will produce files as expected by Excel:mac 2004."
na,the string to use for missing values in the data.
dec,the string to use for decimal points in numeric or complex columns: must be a single character.
row.names,"either a logical value indicating whether the row names of x are to be written along with x, or a character vector of row names to be written."
col.names,"either a logical value indicating whether the column names of x are to be written along with x, or a character vector of column names to be written. See the section on ‘CSV files’ for the meaning of col.names = NA."
